In [ ]:
# import libraries
import pandas as pd
import numpy as np
import math
import datetime as dt
from datetime import timedelta

# charting libraries
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#################
# Definitions
#################
# jeff/AUDUSD_20200212-060000_20200212-180000/2020-02-12T06:00:00.013Z-2020-02-12T18:00:00.866Z-Input_FASTMATCH-EBSDIRECT-REUTERS-HOTSPOT-GTX-CNX-FXALL_top1-1.csv.gz
# Where to read data from
s3_bucket    = 'mfx-sagemaker-dev'

s3_signal_data_key = "demo/XBTUSD_20200809-000000_20200821-120000/2020-08-09T21:01:08.578Z-2020-08-21T12:00:00.982Z-Input_DVCHAIN_top1-1.csv.gz"

s3_signal_data_key = "demo/XBTUSD_20200821-000000_20200910-000000/2020-08-21T00:00:00.609Z-2020-09-10T00:00:00.874Z-Input_DVCHAIN_top1-1.csv.gz"

# Bar size
resample_period = '60t'

# Chart settings
chart_padding_secs = 10

# Breakout settings
breakout_sigma = 2.0

# MR settings
mr_sigma = 3

# Hedging level
hedge_level = 5

# Hedger interest to fill time in ms
interest_to_fill = 10



# n samples for moving average
ma_samples = "120t"

# optimise exits
optimise_exit = True


In [ ]:
#################
# Load from S3
#################

signal_tick_data = pd.read_csv('s3://{}/{}'.format(s3_bucket, s3_signal_data_key), index_col='t', parse_dates=['t'])
trigger_tick_data = signal_tick_data

reval_tick_data = signal_tick_data
# Source for tick charts
chart_tick_data = reval_tick_data

signal_tick_data['spread'] = signal_tick_data['Offer0'] - signal_tick_data['Bid0']
reval_tick_data['spread'] = reval_tick_data['Offer0'] - reval_tick_data['Bid0']




In [ ]:
#################
# Aggregate tick data
#################
signal_mid_price_series = (signal_tick_data.loc[:, 'Bid0'] + signal_tick_data.loc[:, 'Offer0']) / 2
trigger_mid_price_series = (trigger_tick_data.loc[:, 'Bid0'] + trigger_tick_data.loc[:, 'Offer0']) / 2
reval_mid_price_series = (reval_tick_data.loc[:, 'Bid0'] + reval_tick_data.loc[:, 'Offer0']) / 2
# print(reval_mid_price_series.tail())

# reval_mid_price_series['reference_mid'] = signal_mid_price_series.asof(reval_mid_price_series.index)
# print(signal_mid_price_series.asof(reval_mid_price_series.index), reval_mid_price_series)
bar_sampler = signal_mid_price_series.resample(resample_period)

hloc = bar_sampler.ohlc() # **FIXME** First tick in bar not last tick in previous bar?

# display(hloc)

In [ ]:
#################
# Derived columns
#################

hloc['o_to_h'] = (hloc['high'] / hloc['open'] - 1)
hloc['o_to_l'] = (hloc['low'] / hloc['open'] - 1)
hloc['c_to_c'] = hloc['close'].pct_change()

hloc['o_to_h_vol'] = hloc['o_to_h'].rolling(ma_samples).std()
hloc['o_to_l_vol'] = hloc['o_to_l'].rolling(ma_samples).std()
hloc['c_to_c_vol'] = hloc['c_to_c'].rolling(ma_samples).std()






# hloc['6_hr_ma'] = hloc['close'].rolling(6).mean()
# hloc['6_hr_ma_sig'] = np.where(hloc['close'].shift(1) > hloc['6_hr_ma'].shift(1), 1, -1)

In [ ]:
#################
# Trigger check
#################

def mid_trigger_price (timestamp, where): 
    index = trigger_mid_price_series[timestamp : timestamp + timestamp.freq].where(where).dropna().first_valid_index()
    if index == None :
        return None
    result = reval_mid_price_series.asof(index)
#     print(result)
    return result

def trigger_check(row, high_trigger_col, low_trigger_col, high_side):
    
    triggers = [
        ['High', mid_trigger_price(row.name, lambda price: price.gt(row[high_trigger_col])), 
                 +high_side, row[high_trigger_col]],
        ['Low', mid_trigger_price(row.name, lambda price: price.lt(row[low_trigger_col])),
                -high_side, row[low_trigger_col]]
    ]
    
    triggers = [t for t in triggers if t[1] is not None]
    
    triggers.sort(key=lambda t : t[1])
    
    if not triggers:
        return None
    
    return triggers[0]

In [ ]:
#################
# Breakout signals
#################

hloc['breakout_high_trigger'] = hloc['open'] * (1 + (breakout_sigma * (hloc['o_to_h_vol'].shift(1))))
hloc['breakout_low_trigger'] = hloc['open'] * (1 - (breakout_sigma * (hloc['o_to_l_vol'].shift(1))))

hloc['breakout_triggered'] = hloc.apply(lambda row: trigger_check(row, 'breakout_high_trigger', 'breakout_low_trigger', 1), axis=1)
hloc['breakout_triggered'].isna().sum()

In [ ]:
#################
# Mean reversion signals
#################

hloc['6_hr_ma'] = hloc['close'].rolling(ma_samples).mean()
# hloc['6_hr_ma_sig'] = np.where(hloc['close'].shift(1) > hloc['6_hr_ma'].shift(1), 1, -1)

hloc['mr_high_trigger'] = hloc['6_hr_ma'] * (1 + (mr_sigma * (hloc['c_to_c_vol'].shift(1))))
hloc['mr_low_trigger'] = hloc['6_hr_ma'] * (1 - (mr_sigma * (hloc['c_to_c_vol'].shift(1))))

hloc['mr_triggered'] = hloc.apply(lambda row: trigger_check(row, 'mr_high_trigger', 'mr_low_trigger', -1), axis=1)
hloc.tail()

In [ ]:
hloc_filtered = hloc
# .between_time('00:30', '20:00')
# hloc.to_csv('test_4.csv')
# hloc_filtered.to_csv('test_5.csv')


In [ ]:
# position

hloc_filtered['breakout_trade'] = hloc_filtered['breakout_triggered'].map(lambda x: 0 if x is None else x[2])
hloc_filtered['breakout_trade_price'] = hloc_filtered['breakout_triggered'].map(lambda x: 0 if x is None else x[1])
hloc_filtered['breakout_contra_trade_amount'] = -1 * hloc_filtered['breakout_trade'] * hloc_filtered['breakout_trade_price']

hloc_filtered['mr_trade'] = hloc_filtered['mr_triggered'].map(lambda x: 0 if x is None else x[2])
hloc_filtered['mr_trade_price'] = hloc_filtered['mr_triggered'].map(lambda x: 0 if x is None else x[1])
hloc_filtered['mr_contra_trade_amount'] = -1 * hloc_filtered['mr_trade'] * hloc_filtered['mr_trade_price']
hloc_filtered['position'] = hloc_filtered['breakout_trade'].cumsum() + hloc_filtered['mr_trade'].cumsum()



In [ ]:
stdev = np.std(hloc_filtered['close'].diff())

In [ ]:
# hedge trades
hloc_filtered['hedge_trade'] = 0.0
hloc_filtered['hedge_price'] = 0.0
hedge_trade_sum = 0.0
hloc_filtered['hedge_contra_amount'] = 0.0
current_position = 0.0
keep_position = False
i_prev = None
n = len(hloc_filtered)
 
for i in hloc_filtered.index:
    stay_long, stay_short = False, False
    current_position = hloc_filtered.loc[i]['position'] + hedge_trade_sum
    
    if(optimise_exit) :
        if current_position > 0 and (hloc_filtered.loc[i]['close'] > (hloc_filtered.loc[i_prev]['close'] + 2.5 * stdev)):
            stay_long = True
        if current_position < 0 and (hloc_filtered.loc[i]['close'] < (hloc_filtered.loc[i_prev]['close'] - 2.5 * stdev)):
            stay_short = True
        keep_position = stay_long or stay_short
    
    if i_prev and not keep_position and (np.absolute(current_position) >= hedge_level):
        hloc_filtered.loc[i,'hedge_trade'] = -1 * current_position
        hedge_trade_sum += -1 * current_position
        # TODO: Review
        hedge_price = hloc_filtered.asof(i + pd.Timedelta(milliseconds=interest_to_fill),
                                         subset=['close'])['close']
        hloc_filtered.loc[i,'hedge_price'] = hedge_price
        hloc_filtered.loc[i,'hedge_contra_amount'] = -1 * hloc_filtered.loc[i,'hedge_trade'] * \
                                                     hloc_filtered.loc[i,'hedge_price']
    i_prev = i

In [ ]:
hloc_filtered['hedge_balance'] = hloc_filtered['hedge_trade'].cumsum()
hloc_filtered['overall_position'] = hloc_filtered['breakout_trade'].cumsum() + hloc_filtered['mr_trade'].cumsum() + hloc_filtered['hedge_trade'].cumsum()
hloc_filtered['overall_contra_position'] = hloc_filtered['breakout_contra_trade_amount'].cumsum() + hloc_filtered['mr_contra_trade_amount'].cumsum() + hloc_filtered['hedge_contra_amount'].cumsum()

In [ ]:
# #pnl
hloc_filtered['pnl'] = np.where(hloc_filtered['overall_position'] == 0, hloc_filtered['overall_contra_position'], hloc_filtered['overall_position'] * hloc_filtered['close'] + hloc_filtered['overall_contra_position'])
hloc_filtered['pnl_pct'] = hloc_filtered["pnl"] / hloc_filtered["close"]

In [ ]:
# plot mr strategy returns
hloc_filtered['pnl'].resample("60T").last().plot()

In [ ]:
hloc_filtered['c_to_c_vol'].resample("60T").last().plot()

In [ ]:
np.abs(hloc_filtered['overall_position']).mean()

In [ ]:
#volumes, pnl and yield
y = 0
breakout_volume = np.abs(hloc_filtered['breakout_trade']).sum()
mr_volume = np.abs(hloc_filtered['mr_trade']).sum()
hedge_volume = np.abs(hloc_filtered['hedge_trade']).sum()
total_volume = breakout_volume + mr_volume + hedge_volume
total_pnl = hloc_filtered['pnl'].tail(1)
y = total_pnl / total_volume 
display(breakout_volume, mr_volume, hedge_volume, total_volume, total_pnl, y)

In [ ]:
hloc_filtered.tail(20).to_clipboard()

### Order Flow Imbalance

In [ ]:

def ofi(quotes,level):
    """Returns Order Flow Imbalance for given levels of the orderbook"""
    qdf = quotes.copy()
    bid_price_label = 'Bid' + str(level)
    offer_price_label = 'Offer' + str(level)
    bid_qty_label = 'Bid' +str(level) + 'Qty'
    offer_qty_label = 'Offer' + str(level)+'Qty'

    qdf['prev_bidprice'] = qdf[bid_price_label].shift()
    qdf['prev_bidsize'] = qdf[bid_qty_label].shift()
    qdf['prev_askprice'] = qdf[offer_price_label].shift()
    qdf['prev_asksize'] = qdf[offer_qty_label].shift()

    # Fix any missing/invalid data
    qdf.replace([np.inf, np.NINF], np.nan, inplace=True)
    qdf.fillna(method="ffill", inplace=True)
    qdf.fillna(method="bfill", inplace=True)
    
    bid_geq = qdf[bid_price_label] >= qdf['prev_bidprice']
    bid_leq = qdf[bid_price_label] <= qdf['prev_bidprice']
    ask_geq = qdf[offer_price_label] >= qdf['prev_askprice']
    ask_leq = qdf[offer_price_label] <= qdf['prev_askprice']
    
    qdf['ofi'] = np.zeros(len(qdf))
    qdf['ofi'].loc[bid_geq] += qdf[bid_qty_label].loc[bid_geq]
    qdf['ofi'].loc[bid_leq] -= qdf['prev_bidsize'].loc[bid_leq]
    qdf['ofi'].loc[ask_geq] += qdf['prev_asksize'].loc[ask_geq]
    qdf['ofi'].loc[ask_leq] -= qdf[offer_qty_label].loc[ask_leq]    
    return qdf['ofi']




In [ ]:
########################
# OFI using top x levels 
########################

# df_all = pd.read_csv('s3://{}/{}'.format(s3_bucket, s3_signal_data_key), index_col='t', parse_dates=['t'])

# df_all = df_all.resample("100ms").last().ffill()

df_all = signal_tick_data.resample("100ms").last().ffill()
df_all = df_all

df_all["Offer0Qty"] = df_all["Offer0Qty"].astype('float')
df_all["Offer0"] = df_all["Offer0"].astype('float')
df_all["Bid0"] = df_all["Bid0"].astype('float')
df_all["Bid0Qty"] = df_all["Bid0Qty"].astype('float')

df_all["Offer1Qty"] = df_all["Offer1Qty"].astype('float')
df_all["Offer1"] = df_all["Offer1"].astype('float')
df_all["Bid1"] = df_all["Bid1"].astype('float')
df_all["Bid1Qty"] = df_all["Bid1Qty"].astype('float')

df_all["Offer2Qty"] = df_all["Offer2Qty"].astype('float')
df_all["Offer2"] = df_all["Offer2"].astype('float')
df_all["Bid2"] = df_all["Bid2"].astype('float')
df_all["Bid2Qty"] = df_all["Bid2Qty"].astype('float')

df_all["Offer3Qty"] = df_all["Offer3Qty"].astype('float')
df_all["Offer3"] = df_all["Offer3"].astype('float')
df_all["Bid3"] = df_all["Bid3"].astype('float')
df_all["Bid3Qty"] = df_all["Bid3Qty"].astype('float')

df_all["Offer4Qty"] = df_all["Offer4Qty"].astype('float')
df_all["Offer4"] = df_all["Offer4"].astype('float')
df_all["Bid4"] = df_all["Bid4"].astype('float')
df_all["Bid4Qty"] = df_all["Bid4Qty"].astype('float')

In [ ]:
# OFI with levels 1, 2 and 3 (works better than 0,1,2)
# 100 period MA works well

df_all['ofi'] = ofi(df_all,1) +ofi(df_all,2) +ofi(df_all,3) 
df_all['ofi_signal'] = np.where(df_all['ofi'].rolling(200).mean() > 0, 1, -1)
df_all['mid'] = ((df_all['Bid0'] + df_all['Offer0']) / 2.0)
df_all['mid_change'] = ((df_all['Bid0'] + df_all['Offer0']) / 2.0).pct_change()


# shift the signal
df_all['ofi_signal'] = df_all['ofi_signal'].shift(1)
df_all['ofi_pnl'] = (df_all['ofi_signal'] * df_all['mid_change'])

print("Cumulative PnL " + str(df_all['ofi_pnl'].cumsum().iloc[-1]))



In [ ]:
df_all['ofi_pnl'].cumsum().resample("1T").last().plot()

In [ ]:
df_all['ofi_signal'].rolling("90t").sum().resample("1T").last().plot()

In [ ]:
3*90*100000

In [ ]:
ofi_direction = np.sign(df_all['ofi_signal'])

In [ ]:
ofi_direction.sum()

### Volume Weighted Mids

In [ ]:
#######################
# Volume Weighted Mids
#######################

In [ ]:
# CWM
df_all['weighted_bid_notional_5'] =  df_all["Bid1Qty"] + df_all["Bid2Qty"] + df_all["Bid3Qty"] + df_all["Bid4Qty"]
df_all['weighted_offer_notional_5'] =  df_all["Offer1Qty"] + df_all["Offer2Qty"] + df_all["Offer3Qty"] + df_all["Offer4Qty"]
df_all['weighted_bid_5'] = (df_all["Bid1Qty"] * df_all["Bid1"] + df_all["Bid2Qty"] * df_all["Bid2"] + df_all["Bid3Qty"] * df_all["Bid3"] + df_all["Bid4Qty"] * df_all["Bid4"]) / df_all['weighted_bid_notional_5'] 
df_all['weighted_offer_5'] = ( df_all["Offer1Qty"] * df_all["Offer1"] + df_all["Offer2Qty"] * df_all["Offer2"] + df_all["Offer3Qty"] * df_all["Offer3"] + df_all["Offer4Qty"] * df_all["Offer4"]) / df_all['weighted_offer_notional_5'] 
df_all['conventionally_weighted_mid_5'] = (df_all['weighted_bid_5'] + df_all['weighted_offer_5']) / 2
df_all['cwm'] = df_all['conventionally_weighted_mid_5']
df_all['cwm_signal'] = np.where(df_all['cwm'] >= df_all['mid'], 1, -1)


# slow down to XTX pace 
df_all['cwm_signal_slow'] = df_all['cwm_signal'].rolling(10).mean()


# shift signal and calculate returns
df_all['cwm_signal'] = df_all['cwm_signal_slow'].shift(1)
df_all['cwm_pnl'] = df_all['cwm_signal'] * df_all['mid_change']
df_all['cwm_pnl'].cumsum().resample("1T").last().plot()


In [ ]:
# VWM - level 0 inverse
df_all['inverse_vwm'] = (df_all["Bid0Qty"] * df_all["Offer0"] + df_all["Bid0"] * df_all["Offer0Qty"])  / (df_all["Bid0Qty"] + df_all["Offer0Qty"])
df_all['inverse_signal'] = np.where(df_all['inverse_vwm'] >= df_all['mid'], -1, 1)

# shift signal
df_all['inverse_signal'].shift(1)

# calculate returns
df_all['inverse_pnl'] = df_all['inverse_signal'] * df_all['mid_change']
df_all['inverse_pnl'].cumsum().resample("1T").last().plot()

In [ ]:
#VWM - level 1 
df_all['vwm'] = (df_all["Bid1Qty"] * df_all["Offer1"] + df_all["Bid1"] * df_all["Offer1Qty"])  / (df_all["Bid1Qty"] + df_all["Offer1Qty"])
df_all['vwm_signal'] = np.where(df_all['vwm'] >= df_all['mid'], 1, -1)

df_all['vwm_signal'] = np.where(df_all['vwm'] > df_all['mid'], 1, -1)

# shift the signal
df_all['vwm_signal'] = df_all['vwm_signal'].shift(1)

# calculate returns
df_all['vwm_pnl'] = df_all['vwm_signal'] * df_all['mid_change']
df_all['vwm_pnl'].cumsum().resample("1T").last().plot()

### Trend Following

In [ ]:
####################
# Trend and Crossover
####################

# simple trend following model for mid generation
df_all['trend_signal'] = np.where(df_all['mid'] > df_all['mid'].rolling(75).mean(), 1, -1)
df_all['trend_signal'] = df_all['trend_signal'].shift(1)
df_all['trend_return'] = df_all['trend_signal'] * df_all['mid_change']

# ma crossover
df_all['crossover_signal'] = np.where(df_all['mid'].rolling(100).mean() > df_all['mid'].rolling(500).mean(), 1, -1)
df_all['crossover_signal'] = df_all['crossover_signal'].shift(1)
df_all['crossover_return'] = df_all['crossover_signal'] * df_all['mid_change']


In [ ]:
df_all[['crossover_return', 'trend_return']].between_time('00:00', '22:00').cumsum().resample("1T").last().plot()
# df_all[['crossover_return', 'trend_return']].cumsum().resample("1T").last().plot()

In [ ]:
df_all['combined_signal'] = np.sign(df_all['ofi_signal'] + df_all['trend_signal'] + df_all['cwm_signal'])
df_all['combined_pnl'] = df_all['mid_change'] * df_all['combined_signal']
df_all['combined_pnl'].between_time('07:00', '19:00').cumsum().resample("1T").last().plot()

### Adapt signal - Regime Switching 

#### Step 1 - slow the signals down so they have the same expected flip rate (5 seconds)
#### Step 2 - calculate pnl moving average for each
#### Step 3 - weight according to each distance to moving average
#### make sure the shift is done

In [ ]:
df_all['ofi_pnl_ma'] = df_all['ofi_pnl'].rolling(1000).mean()
df_all['trend_pnl_ma'] = df_all['trend_return'].rolling(1000).mean()
df_all['cwm_pnl_ma'] = df_all['cwm_pnl'].rolling(1000).mean()

In [ ]:
df_all['combined_pnl_ma'] = (df_all['ofi_pnl_ma'] + df_all['trend_pnl_ma'] + df_all['cwm_pnl_ma']) / 3

In [ ]:
df_all['ofi_weight'] = df_all['ofi_pnl_ma'] / df_all['combined_pnl_ma']
df_all['ofi_weight'] = np.max(df_all['ofi_weight'], 0)
df_all['ofi_weight'] = df_all['ofi_weight'].shift(1)

In [ ]:
df_all['trend_weight'] = df_all['trend_pnl_ma'] / df_all['combined_pnl_ma']
df_all['trend_weight'] = np.max(df_all['trend_weight'], 0)
df_all['trend_weight'] = df_all['trend_weight'].shift(1)

In [ ]:
df_all['cwm_weight'] = df_all['cwm_pnl_ma'] / df_all['combined_pnl_ma']
df_all['cwm_weight'] = np.max(df_all['cwm_weight'], 0)
df_all['cwm_weight'] = df_all['cwm_weight'].shift(1)

In [ ]:
df_all['adapt_signal'] = np.sign(
    (df_all['cwm_weight'] * df_all['cwm_signal']) + 
    (df_all['trend_weight'] * df_all['trend_signal']) +
    (df_all['ofi_weight'] * df_all['ofi_signal'])
)

In [ ]:
df_all['adapt_pnl'] = df_all['mid_change'] * df_all['adapt_signal']

In [ ]:
df_all['adapt_pnl'].cumsum().resample("1T").last().plot()

In [ ]:
df_all['60change'] = (df_all['mid'].shift(-3600) / df_all['mid'])-1

In [ ]:
df_all['60pnl'] = df_all['adapt_signal'] * df_all['60change']

In [ ]:
df_all['60pnl'].cumsum().resample("1T").last().plot()

In [ ]:
df_all.head()

### Pnl Per Trade and Holding Period

In [ ]:
###############
# Counts the number of ticks between changes in the signal
# Calculates pnl per trade 
###############

# these are the df and column for the signals 
signal_df = df_all
signal_column = 'adapt_signal'

# the df and column for the pnl
pnl_column = df_all['adapt_pnl']


def SignalPersisenceFast(df,column_name): 
    array= df[column_name].values
    previous_signal  = False 
    Counter = 0
    Times = []
    for x in range(len(array)):
        if((array[x] == previous_signal or Counter == 0) and array[x] != 0):
            Counter = Counter + 1
        else:
            Times.append(Counter)
            if array[x] != 0 : 
                Counter =  1
        previous_signal = array[x]
    return Times

Times = SignalPersisenceFast(df_all,signal_column)
print("Mean time in trade " + str(np.mean(Times)))
print("Number of trades " + str(len(Times)))
print("Cumulative PnL " + str(pnl_column.cumsum().iloc[-1]))
print("Average Trade PnL " + str(pnl_column.cumsum().iloc[-1] / len(Times)))

# Horizons

In [ ]:
df_all['ofi_signal_10'] = np.where(df_all['ofi'].rolling(10).mean() > 0, 1, -1)
df_all['ofi_signal_25'] = np.where(df_all['ofi'].rolling(25).mean() > 0, 1, -1)
df_all['ofi_signal_50'] = np.where(df_all['ofi'].rolling(50).mean() > 0, 1, -1)
df_all['ofi_signal_100'] = np.where(df_all['ofi'].rolling(100).mean() > 0, 1, -1)
df_all['ofi_signal_200'] = np.where(df_all['ofi'].rolling(200).mean() > 0, 1, -1)

In [ ]:
df_all['trend_sig_5'] = np.where(df_all['mid'] > df_all['mid'].rolling(5).mean(), 1, -1)
df_all['trend_sig_10'] = np.where(df_all['mid'] > df_all['mid'].rolling(10).mean(), 1, -1)
df_all['trend_sig_25'] = np.where(df_all['mid'] > df_all['mid'].rolling(25).mean(), 1, -1)
df_all['trend_sig_50'] = np.where(df_all['mid'] > df_all['mid'].rolling(50).mean(), 1, -1)
df_all['trend_sig_100'] = np.where(df_all['mid'] > df_all['mid'].rolling(100).mean(), 1, -1)

In [ ]:
df_all_filtered = df_all.between_time('09:00', '12:00')


In [ ]:
fig = go.Figure(go.Scatter(
    x = df_all_filtered.index,
    y = df_all_filtered['mid']
))

fig.update_layout(
    xaxis_tickformat = '%H:%M:%S.%L'
)

fig.update_layout(
    yaxis_tickformat = '.5f'
)

fig.show()

In [ ]:
# Create traces
trace0 = go.Scatter(
    x = df_all_filtered.index,
    y = df_all_filtered['trend_sig_25'],
    mode = 'lines',
    name = '25'
)
trace1 = go.Scatter(
    x = df_all_filtered.index,
    y = df_all_filtered['trend_sig_50'],
    mode = 'lines',
    name = '50'
)
trace2 = go.Scatter(
    x = df_all_filtered.index,
    y = df_all_filtered['trend_sig_100'],
    mode = 'lines',
    name = '100'
)


data = [trace2]

py.iplot(data, filename='line-mode')